# Merging 2 - products_clean and orders_products_combined

### List of contents
    1. Importing file
	2. Frequency checks before merging
	3. Creating a version of dataframe combine without product_id 26520 and 6800
	4. Merging files
	5. Missing values?
    6. Dropping unwanted columns
	7. Exporting 

## Importing libraries

In [4]:
import pandas as pd
import numpy as np
import os

## Creating path for folder

In [7]:
path = r'/Users/ceciliamoura/Desktop/Career Foundry/Achievement4/Instacart Basket Analysis'

## 1. Importing files

In [10]:
# Importing products_clean

In [12]:
prods = pd.read_csv(os.path.join (path, 'Data', 'Final Prepared Data', 'prods_clean.csv'), index_col = False)

In [14]:
prods.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3


In [16]:
prods.shape

(49672, 6)

In [20]:
# Importing orders_products_combined

combined = pd.read_pickle(os.path.join (path, 'Data', 'Final Prepared Data', 'orders_products_combined.pkl'))

In [22]:
combined.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_day_of_week,order_time,days_since_prior_order
0,2,33120,1,1,202279,3,5,9,8
1,2,28985,2,1,202279,3,5,9,8
2,2,9327,3,0,202279,3,5,9,8
3,2,45918,4,1,202279,3,5,9,8
4,2,30035,5,0,202279,3,5,9,8


In [24]:
combined.shape

(32434489, 9)

## 2. Frequency checks before merging

In [29]:
#performing frequency check
prods['product_id'].value_counts(dropna = False)

product_id
26520    2
6800     2
1        1
33136    1
33127    1
        ..
16576    1
16577    1
16578    1
16579    1
49688    1
Name: count, Length: 49670, dtype: int64

In [31]:
# product_id from prods file should be unique values

### 2.1 Product_ids appearing two times - 26520 and 6800

In [49]:
dup_prods = pd.concat([prods[prods['product_id'] == 26520], prods[prods['product_id'] == 6800]])

In [51]:
dup_prods

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
26504,26520,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26505,26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9
6784,6799,6800,Revive Zero Vitamin Water,64,7,6.4
6785,6800,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0


In [53]:
# conclusion: product_id 26520 and  6800 refer each to two different products, which will cause error in analysis

In [87]:
dup_prods.to_clipboard()

### 2.2 Total occurrence of product_id 26520 and 6800 in orders_products_combined

In [60]:
dup_combined = pd.concat([combined[combined['product_id'] == 26520], combined[combined['product_id'] == 6800]])

In [62]:
dup_combined

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_day_of_week,order_time,days_since_prior_order
22145,2334,26520,5,0,111317,8,4,10,3
63533,6775,26520,3,0,46374,13,1,10,17
75924,8074,26520,9,0,98750,7,0,18,20
140100,14757,26520,3,1,161131,14,0,13,6
181438,19108,26520,4,1,161131,28,6,10,11
...,...,...,...,...,...,...,...,...,...
20249285,2135835,6800,18,0,139099,20,2,14,7
22256152,2347508,6800,12,1,175905,4,5,10,17
24049891,2536418,6800,16,1,175905,5,6,8,1
27549987,2905190,6800,34,0,175905,3,2,11,27


In [64]:
combined.shape

(32434489, 9)

In [93]:
# 570 rows out of 32434489 contain data with ambiguous product_id == 0.001% of full data
# coclusion = it is not a representative number
# decision = will delete product_id 26520 and 6800 from 'combined' file before merging. 
# Since I will do an inner merge, there won't be any observation relating to ambiguous product_id 

## 3. Creating a version of dataframe combine without product_id 26520 and 6800

In [83]:
combined_no_dups = combined[~(combined['product_id'] == 26520) & ~(combined['product_id'] == 6800)]

In [89]:
combined_no_dups.shape

(32433919, 9)

In [91]:
#exactly 570 less rows than 'combined' dataframe

## 4. Merging files

In [117]:
# key: product_id
# first dataframe = combined_no_dups
# second dataframe = prods
# combined dataframe = ords_prods_merged

ords_prods_merged = combined_no_dups.merge(prods, on = 'product_id', how = 'inner', indicator = True)

In [119]:
#checking new merged file
ords_prods_merged.shape

(32403719, 15)

In [121]:
ords_prods_merged.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_day_of_week,order_time,days_since_prior_order,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge
0,2,33120,1,1,202279,3,5,9,8,33122,Organic Egg Whites,86,16,11.3,both
1,2,28985,2,1,202279,3,5,9,8,28987,Michigan Organic Kale,83,4,13.4,both
2,2,9327,3,0,202279,3,5,9,8,9327,Garlic Powder,104,13,3.6,both
3,2,45918,4,1,202279,3,5,9,8,45922,Coconut Butter,19,13,8.4,both
4,2,30035,5,0,202279,3,5,9,8,30037,Natural Sweetener,17,13,13.7,both


In [123]:
# Checking _merge flag

ords_prods_merged['_merge'].value_counts()

_merge
both          32403719
left_only            0
right_only           0
Name: count, dtype: int64

## 5. Missing values?

In [125]:
ords_prods_merged.isnull().sum()

order_id                     0
product_id                   0
add_to_cart_order            0
reordered                    0
user_id                      0
order_number                 0
order_day_of_week            0
order_time                   0
days_since_prior_order       0
Unnamed: 0                   0
product_name                 0
aisle_id                     0
department_id                0
prices                    5127
_merge                       0
dtype: int64

In [127]:
# Creating a subset with missing values

prices_missing = ords_prods_merged[ords_prods_merged['prices'].isnull() == True]

In [129]:
prices_missing

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_day_of_week,order_time,days_since_prior_order,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge
347,43,21553,6,1,39630,14,5,17,4,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
9177,950,21553,20,0,187854,16,1,11,16,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
10798,1115,21553,14,1,168014,15,6,14,30,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
11147,1155,21553,18,1,103747,22,1,13,12,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
11769,1220,21553,19,0,23475,3,0,17,26,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32398648,3420548,21553,5,1,146887,3,4,11,7,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
32399191,3420604,21553,8,0,159443,8,2,13,6,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
32399852,3420676,21553,10,0,202268,50,5,16,6,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both
32401335,3420827,21553,2,0,15643,1,1,10,0,21554,Lowfat 2% Milkfat Cottage Cheese,108,16,NaN,both


In [133]:
# These probably refer to the two products flagged as abnormally high-priced products
    #product_id 21553 and product_id 33664

In [143]:
prices_missing['product_id'].value_counts()

product_id
21553    4429
33664     698
Name: count, dtype: int64

In [145]:
# all missing prices refer to products previously flagged with abnormal price and turned into NaN intencionally

## 6. Dropping unwanted columns

In [147]:
ords_prods_merged = ords_prods_merged.drop(columns = ['Unnamed: 0', '_merge'])

In [151]:
# checking procedure

ords_prods_merged.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
       'order_number', 'order_day_of_week', 'order_time',
       'days_since_prior_order', 'product_name', 'aisle_id', 'department_id',
       'prices'],
      dtype='object')

In [153]:
ords_prods_merged.shape

(32403719, 13)

In [155]:
ords_prods_merged.dtypes

order_id                    int64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
user_id                     int64
order_number                int64
order_day_of_week           int64
order_time                  int64
days_since_prior_order      int64
product_name               object
aisle_id                    int64
department_id               int64
prices                    float64
dtype: object

In [118]:
# I will switch columns: order_id, product_id and user_id into object after final merge. 

## 7. Exporting combined file

In [157]:
ords_prods_merged.to_pickle(os.path.join(path, 'Data','Final Prepared Data', 'orders_products_merged.pkl'))